In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np


def mplot(img,img2=None):
    
    cv2.namedWindow('img',cv2.WINDOW_NORMAL)
    cv2.imshow('img',img)
    if img2 is not None:
        cv2.namedWindow('img2',cv2.WINDOW_NORMAL)
        cv2.imshow('img2',img2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [100]:
'''
function to rearrange the contour bounding boxes. in default the contour bounding boxes comes in the sorted order of
their y co-ordinates . this function returns a list of rectangles [(x1,y1,w1,h1),(x2,y2,w2,h2)...] which are sorted in
the order of x axis on each line. a line will have all recangles of y coordinates between y and y+h of first rectangle '''
def rearrange(cnt):
    b_rect=[]
    for c in cnt:
        rect=cv2.boundingRect(c)
        if rect[2] <=20 or rect[3] <= 20:
            continue
        b_rect.append(rect)
    if b_rect==[]:
        return []
    p=b_rect[0][1]+b_rect[0][3]
    #print('length of brect:',len(b_rect))
    s_rect=[]
    i=0
    length=len(b_rect)
    while i<length:
        p=b_rect[i][1]+b_rect[i][3]
        elem_on_line=[]#elements on a line
        outer=True
        while i<length and p>b_rect[i][1]:
            elem_on_line.append(b_rect[i])
            i+=1
            outer=False
        if outer:
            i+=1
        elem_on_line=sorted(elem_on_line) #,key=lambda x:x[0]
        #print(elem_on_line,i)
        s_rect.extend(elem_on_line)
    return s_rect
        

In [3]:
'''
clear_noice method clear noice from list of images and retrun cleared list of images
'''
def clear_noice(image,ellipse=(7,7),rect=(13,7)):
    
    threshold=128
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, ellipse )
    kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, rect )
    cnt=[]
    grad = cv2.morphologyEx(image, cv2.MORPH_GRADIENT, kernel)
    _, bw = cv2.threshold(grad, threshold, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel2)
    _,contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL ,cv2.CHAIN_APPROX_SIMPLE )
    #reversing contour list to start processig from top
    contours.reverse()
    cnt.extend(contours)
    #print('contour length:',len(contours))
    #mplot(grad,connected)
    return cnt

In [102]:
img=cv2.imread('/home/jabir/Project/MHCR/realtest/img5.jpg',0)
img2=img.copy()
contours=clear_noice(img) # it expects a list of images so passing single image as list 
#print(len(contours))

In [103]:
#cutting the image into list of words
s_rect=rearrange(contours)
words=[]
i=0
#print("length of rect",len(s_rect))
for rect in s_rect:
    x,y,w,h = rect
    cv2.rectangle(img2,(x,y),(x+w,y+h),(0,0,0),2)
    cv2.putText(img2,str(i),(x+w+10,y+h),0,0.3,(0,0,0))
    word=(img[y:y+h,x:x+w])
    ret,thresh4 = cv2.threshold(word,127,255,cv2.THRESH_TOZERO)
    word=np.array([[min(j+55,255) if j>130 else j for j in thresh4[k]] for k in range(len(thresh4))],dtype=np.uint8)
    words.append(word)
    i+=1
mplot(img2)

<h4> Drawing bounding boxes on each words</h4><br/>
<p> Here, word consists of list of words cropped.we are rearranging each word and drawing bounding rectangle to each characters  </p>

In [106]:
seperated=[]
for i in range(len(words)):
    
    c=clear_noice(words[i],ellipse=(3,3),rect=(2,2))
    s_rect=rearrange(c)
    #print("rect:",s_rect)
    chars=[]
    if s_rect==[]:
        continue
    for rect in s_rect:
        x,y,w,h = rect
        char=words[i][y:y+h,x:x+w]
        char= cv2.copyMakeBorder(char,20,20,10,10,cv2.BORDER_CONSTANT,value=[255,255,255])
        char=cv2.resize(char,(86,86))
        chars.append(char)
    #print(chars)
        #cv2.rectangle(words[i],(x,y),(x+w,y+h),(0,0,0),2)
        #cv2.putText(words[i],str(i),(x+w+10,y+h),0,0.3,(0,0,0))
           
    #mplot(words[i])
    seperated.append(chars)
    

In [107]:
for i,word in enumerate(seperated):
    for j,char in enumerate(word):
        '''ret,thresh3 = cv2.threshold(char,127,255,cv2.THRESH_TRUNC)
        ret,thresh4 = cv2.threshold(char,127,255,cv2.THRESH_TOZERO)
        ret,thresh5 = cv2.threshold(char,127,255,cv2.THRESH_TOZERO_INV)
        mplot(thresh4)'''
        mplot(char)

In [148]:
roi=thresh4
nroi=np.array([[min(j+10,255)] if j>200 else j for j in roi[i]] for i in range(len(roi))],dtype=np.uint8)

SyntaxError: invalid syntax (<ipython-input-148-202fcb670497>, line 2)

In [20]:
import numpy as np
from numpy import random
import keras
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import load_model

Using TensorFlow backend.


In [31]:
model2=load_model('/home/jabir/Project/MHCR/modelMHCR_gray_2.8.96.97.h5')

In [97]:
np.array(words)
words[0].shape

(44, 720)

In [44]:
#pred=model2.predict_class
for i,word in enumerate(seperated):
    if word ==[]:
        continue
    word=np.array(word)
    #print(word.shape)
    pred=model2.predict_classes(word.reshape(-1,86,86,1))
    print('nextword')
    for j in pred:
        print(j+1)
    #for j,char in enumerate(word):
       # print(char.shape)
        
        #mplot(char)

nextword
16
nextword
16
nextword
16
nextword
16
nextword
16
nextword
2
9
nextword
9
nextword
16
9
nextword
21


In [13]:
s=seperated[0][0]
char= cv2.copyMakeBorder(char,20,20,10,10,cv2.BORDER_CONSTANT,value=[255,255,255])
#s.shape
pred=model2.predict_classes(s.reshape(-1,86,86,1))
print(pred)

[39 26]


In [41]:
mplot(seperated[1][0])

In [49]:
seperated

[[], [array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         ...,
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)], [array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         ...,
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)], [array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         ...,
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)], [array([[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 

In [88]:
for i in words:
    mplot(i)

In [83]:
words[0].shape


(44, 720)